In [7]:
import numpy as np
import netCDF4 as nc
from pathlib import Path
import os
import glob
import argparse
import pandas as pd
import math


In [55]:
def get_result_data(dataset, xlsx_data,output_path, diff=0.01):
    data = dataset.variables
    lat_mat, lon_mat,elevation_mat =  data['lat'][:].data, data['lon'][:].data,data['elevation'][:].data
    dst_data = xlsx_data.to_numpy()
    diff  = 0.01

    result = "lat,lon,real_lat,real_lon,lat_i, lon_i, distance,elevation"

    for _lon,_lat in dst_data:
        lon_s,lon_e = _lon- diff, _lon + diff
        lat_s,lat_e = _lat - diff, _lat + diff
        lon_mask = np.where((lon_mat >= lon_s) & (lon_mat <= lon_e), np.ones_like(lon_mat), np.zeros_like(lon_mat))
        lat_mask = np.where((lat_mat >= lat_s) & (lat_mat <= lat_e), np.ones_like(lat_mat), np.zeros_like(lat_mat))
        
        lat_i = np.where(lat_mask > 0)[0]
        lon_i = np.where(lon_mask > 0)[0]
        
        real_lon, real_lat, min_dis,elevation = -1, -1, math.inf,-1
        for _lat_i in lat_i:
            for _lon_i in lon_i:
                _lat_d, _lon_d = lat_mat[_lat_i], lon_mat[_lon_i]
                dd = math.sqrt((_lat_d - _lat)**2 + (_lon_d - _lon)**2)
                if dd < min_dis:
                    min_dis = dd
                    real_lat, real_lon = _lat_d,_lon_d
                    elevation = elevation_mat[_lat_i, _lon_i]
        _new_line = f"{_lat}, {_lon},{real_lat},{real_lon}, {_lat_i},{_lon_i}, {min_dis},{elevation}"
        
        result += "\n" + _new_line

    with open(output_path, 'w', encoding='utf8') as f:
        f.write(result)


In [58]:
nc_f_path = "TASK06-20241003/gebco_2024_n33.0_s26.0_w120.0_e127.0.nc"
xlsx_f_path = "TASK06-20241003/data_v2.xlsx"

dataset = nc.Dataset(nc_f_path)
xlsx_data = pd.read_excel(xlsx_f_path)

output_path = "TASK06-20241003/data_v2_result_v1.csv"

In [59]:
get_result_data(dataset=dataset, xlsx_data=xlsx_data, output_path=output_path)